In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

plt.style.use("ggplot")

In [63]:
def noAccent(text):
    accent = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ü": "u"
    }
    text = text
    for k,v in accent.items():
        text = text.replace(k, v)
    return text

# Items 2024

In [64]:
df = pd.read_csv("./DATA/Items2024.csv", sep=";")

df.head()

,CodigoItem,DescripcionItem,PrecioItem,CantidadItem,Municipio,Latitud (Y),Longitud (X),Fecha de Ejecución
0,1.1,Canalización tubería de 25 mm en asfalto (FVA),109834.0,1.0,BELLO,6.351080,-75.518336,29.08.2024 0:00
1,1.1,Canalización tubería de 25 mm en asfalto (FVA),109834.0,10.0,ITUANGO,7.064548,-75.683927,06.12.2024 0:00
2,1.1.13,Canalización tubería de 25 mm en andén,55097.0,2.0,CAREPA,7.847447,-76.740111,12.08.2024 0:00
3,1.1.13,Canalización tubería de 25 mm en andén,55097.0,13.6,TURBO,7.930575,-76.623300,27.08.2024 0:00
4,1.1.13,Canalización tubería de 25 mm en andén,55097.0,25.8,CAREPA,7.759252,-76.652718,18.09.2024 0:00


In [65]:
newDF = {
    "CodigoItem": list(),
    "Item": list(),
    "Cantidad": list(),
    "Municipio": list(),
    "Y": list(),
    "X": list(),
    "FechaEjec": list(),
    "Demolicion": list(),
    "Excavacion": list()
}

cantRCD = {
    "Acometida en pavimento flexible": (0.2*0.2, 0.25*0.2),
    "Acometida en pavimento rigido": (0.2*0.2, 0.25*0.2),
    "Acometida en piso duro": (0.2*0.2, 0.25*0.2),
    "Acometida en zona verde": (0, 0.45*0.2),
    "Acometida en zona verde o destapado": (0, 0.45*0.2),
    "Nicho en anden": (0.2*0.5*0.8, 0.4*0.5*0.8),
    "Nicho y empalme en pavimento flexible": (0.2*0.5*0.8, 0.4*0.5*0.8),
    "Nicho y empalme en piso duro": (0.2*0.5*0.8, 0.4*0.5*0.8),
    "Nicho y empalme en zona verde": (0, 0.6*0.5*0.8)
}

for r in range(0, len(df), 1):
    item = noAccent(" ".join([i for i in df["DescripcionItem"][r].split(" ") if (i != "") and (i != "(FVA)")]))
    if not (df["DescripcionItem"][r].split(" ")[0] in ["S.T.I.", "STI", "Accesorios"]) and not (item in ["Acometida comercial cualquier acabado", "Acometida tuneliada"]) and df["Latitud (Y)"][r] != np.nan:
        newDF["CodigoItem"].append(df["CodigoItem"][r])
        newDF["Item"].append(item)
        newDF["Cantidad"].append(df["CantidadItem"][r])
        newDF["Municipio"].append(df["Municipio"][r])
        newDF["Y"].append(df["Latitud (Y)"][r])
        newDF["X"].append(df["Longitud (X)"][r])
        newDF["FechaEjec"].append(df["Fecha de Ejecución"][r])
        if "Canalizacion" in item:
            if int(item.split(" ")[3]) > 90:
                ancho = 0.35
            else:
                ancho = 0.25
            if item.split("en")[1].replace(" ", "") in ["zonaverde", "destapado"]:
                newDF["Demolicion"].append(0)
                newDF["Excavacion"].append(ancho*(0.6 + int(item.split(" ")[3])/10)*df["CantidadItem"][r])
            elif item.split("en")[1].replace(" ", "") in ["concreto", "asfalto"]:
                newDF["Demolicion"].append(ancho*0.15*df["CantidadItem"][r])
                newDF["Excavacion"].append(ancho*(0.6 - 0.15 + int(item.split(" ")[3])/10)*df["CantidadItem"][r])
            elif item.split("en")[1].replace(" ", "") in ["and", "cuneta", "adoquin"]:
                newDF["Demolicion"].append(ancho*0.1*df["CantidadItem"][r])
                newDF["Excavacion"].append(ancho*(0.6 - 0.1 + int(item.split(" ")[3])/10)*df["CantidadItem"][r])
            else:
                print(item.split("en")[1])
                print(item)
        elif "Acometida" in item:
            newDF["Demolicion"].append(cantRCD[item][0]*df["CantidadItem"][r])
            newDF["Excavacion"].append(cantRCD[item][1]*df["CantidadItem"][r])
        else:
            newDF["Demolicion"].append(cantRCD[item][0])
            newDF["Excavacion"].append(cantRCD[item][1])

for k, v in newDF.items():
    print(k, len(v))

CodigoItem 11323
Item 11323
Cantidad 11323
Municipio 11323
Y 11323
X 11323
FechaEjec 11323
Demolicion 11323
Excavacion 11323


In [66]:
pd.DataFrame(newDF).to_csv("./DATA/ItemsEjec.csv", index=False)